In [1]:
import pandas as pd
import numpy as np
import string
import unicodedata
import gensim
from gensim.utils import simple_tokenize
import stop_words
import random
from sklearn.metrics import log_loss, accuracy_score
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.models import load_model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Load The Data

In [2]:
data= pd.read_csv('./data/isear.csv',error_bad_lines=False,delimiter='|')
print data.shape
data.head()

(7503, 43)


Skipping line 199: expected 43 fields, saw 44
Skipping line 222: expected 43 fields, saw 44
Skipping line 248: expected 43 fields, saw 44
Skipping line 350: expected 43 fields, saw 44
Skipping line 395: expected 43 fields, saw 44
Skipping line 423: expected 43 fields, saw 44
Skipping line 480: expected 43 fields, saw 44
Skipping line 552: expected 43 fields, saw 44
Skipping line 601: expected 43 fields, saw 45
Skipping line 604: expected 43 fields, saw 45
Skipping line 612: expected 43 fields, saw 45
Skipping line 722: expected 43 fields, saw 44
Skipping line 765: expected 43 fields, saw 44
Skipping line 800: expected 43 fields, saw 46
Skipping line 818: expected 43 fields, saw 45
Skipping line 891: expected 43 fields, saw 44
Skipping line 928: expected 43 fields, saw 44
Skipping line 963: expected 43 fields, saw 44
Skipping line 965: expected 43 fields, saw 45
Skipping line 974: expected 43 fields, saw 44
Skipping line 985: expected 43 fields, saw 44
Skipping line 1064: expected 43 fi

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,RELA,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE,Unnamed: 42
0,11001,1,1,1,1,33,1,2,6,1,...,3,2,0,joy,4,3,110011,"During the period of falling in love, each tim...",1,NaN
1,11001,1,1,1,1,33,1,2,6,1,...,2,0,0,fear,3,2,110012,When I was involved in a traffic accident.,1,NaN
2,11001,1,1,1,1,33,1,2,6,1,...,1,0,0,anger,1,3,110013,When I was driving home after several days of...,1,NaN
3,11001,1,1,1,1,33,1,2,6,1,...,1,0,2,sadness,4,4,110014,When I lost the person who meant the most to me.,1,NaN
4,11001,1,1,1,1,33,1,2,6,1,...,2,0,0,disgust,4,4,110015,The time I knocked a deer down - the sight of ...,1,NaN


In [3]:
def category(s):
    if s=='shame':
        return 0
    elif s=='sadness': 
        return 1
    elif s=='fear':
        return 2
    elif s=='anger':
        return 3
    elif s=='guilt':
        return 4
    elif s=='joy':
        return 5
    elif s=='disgust':
        return 6
data['Class']= data['Field1'].map(category)
data.head()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE,Unnamed: 42,Class
0,11001,1,1,1,1,33,1,2,6,1,...,2,0,joy,4,3,110011,"During the period of falling in love, each tim...",1,NaN,5
1,11001,1,1,1,1,33,1,2,6,1,...,0,0,fear,3,2,110012,When I was involved in a traffic accident.,1,NaN,2
2,11001,1,1,1,1,33,1,2,6,1,...,0,0,anger,1,3,110013,When I was driving home after several days of...,1,NaN,3
3,11001,1,1,1,1,33,1,2,6,1,...,0,2,sadness,4,4,110014,When I lost the person who meant the most to me.,1,NaN,1
4,11001,1,1,1,1,33,1,2,6,1,...,0,0,disgust,4,4,110015,The time I knocked a deer down - the sight of ...,1,NaN,6


In [4]:
data['STATE'].value_counts()

1    7503
Name: STATE, dtype: int64

In [5]:
data['Field1'].value_counts()

shame      1081
sadness    1080
fear       1079
anger      1070
guilt      1070
joy        1065
disgust    1058
Name: Field1, dtype: int64

In [69]:
#Lets look at the data
s=data[data['Field1']=='joy']['SIT'][7]
print s

When I got a letter offering me the Summer job that I had applied  for


# Cleaning

In [68]:
def remove_unicode(s):
    printable = set(string.printable)
    return filter(lambda x: x in printable, s).rstrip()
data['SIT']=data['SIT'].map(remove_unicode)
data.head()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE,Unnamed: 42,Class
0,11001,1,1,1,1,33,1,2,6,1,...,2,0,joy,4,3,110011,During the period of falling in love each time...,1,NaN,5
1,11001,1,1,1,1,33,1,2,6,1,...,0,0,fear,3,2,110012,When I was involved in a traffic accident,1,NaN,2
2,11001,1,1,1,1,33,1,2,6,1,...,0,0,anger,1,3,110013,When I was driving home after several days of...,1,NaN,3
3,11001,1,1,1,1,33,1,2,6,1,...,0,2,sadness,4,4,110014,When I lost the person who meant the most to me,1,NaN,1
4,11001,1,1,1,1,33,1,2,6,1,...,0,0,disgust,4,4,110015,The time I knocked a deer down the sight of t...,1,NaN,6


In [8]:
def remove_punct(s):
    return s.translate(None, string.punctuation)
data['SIT']=data['SIT'].map(remove_punct)
data.head()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE,Unnamed: 42,Class
0,11001,1,1,1,1,33,1,2,6,1,...,2,0,joy,4,3,110011,During the period of falling in love each time...,1,NaN,5
1,11001,1,1,1,1,33,1,2,6,1,...,0,0,fear,3,2,110012,When I was involved in a traffic accident,1,NaN,2
2,11001,1,1,1,1,33,1,2,6,1,...,0,0,anger,1,3,110013,When I was driving home after several days of...,1,NaN,3
3,11001,1,1,1,1,33,1,2,6,1,...,0,2,sadness,4,4,110014,When I lost the person who meant the most to me,1,NaN,1
4,11001,1,1,1,1,33,1,2,6,1,...,0,0,disgust,4,4,110015,The time I knocked a deer down the sight of t...,1,NaN,6


In [9]:
#Lower and remove stopwords
def remove_stop_words_tokenize(s):
    stop_= stop_words.get_stop_words('en')
    token=list(simple_tokenize(s.lower()))
    token_= []
    for w in token:
        if w not in stop_:
            token_.append(w)

    return ' '.join(token_)
# data['SIT']=data['SIT'].map(remove_stop_words_tokenize)

data.head()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE,Unnamed: 42,Class
0,11001,1,1,1,1,33,1,2,6,1,...,2,0,joy,4,3,110011,During the period of falling in love each time...,1,NaN,5
1,11001,1,1,1,1,33,1,2,6,1,...,0,0,fear,3,2,110012,When I was involved in a traffic accident,1,NaN,2
2,11001,1,1,1,1,33,1,2,6,1,...,0,0,anger,1,3,110013,When I was driving home after several days of...,1,NaN,3
3,11001,1,1,1,1,33,1,2,6,1,...,0,2,sadness,4,4,110014,When I lost the person who meant the most to me,1,NaN,1
4,11001,1,1,1,1,33,1,2,6,1,...,0,0,disgust,4,4,110015,The time I knocked a deer down the sight of t...,1,NaN,6


In [10]:
data['SIT'].values

array([ 'During the period of falling in love each time that we met and  especially when we had not met for a long time',
       'When I was involved in a traffic accident',
       'When I was driving home after  several days of hard work there  was a motorist ahead of me who was driving at 50 kmhour and  refused despite his low speeed to let me overtake',
       ...,
       'I was at home and I heard a loud sound of spitting outside the  door  I thought that one of my family members would step on the spit  and bring the germs in the house',
       'I did not do the homework that the teacher had asked us to do  I  was scolded immediately',
       'I had shouted at my younger brother and he was always afraid when  I called out loudly'], dtype=object)

# Model Build

In [59]:
#Apply Simple Keras
num_words=1000
tokenizer= Tokenizer(num_words=num_words,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',lower=True,
                    split=' ')
tokenizer.fit_on_texts(data['SIT'].values)
 

In [60]:
# Pad the data 
X = tokenizer.texts_to_sequences(data['SIT'].values)
X = pad_sequences(X, maxlen=1000)

In [61]:
X.shape[1]

1000

In [62]:
#Build Our Simple LSTM
embed_dim = 128
lstm_out = 196

# Model saving callback
ckpt_callback = ModelCheckpoint('keras_model', 
                                 monitor='val_loss', 
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')

model = Sequential()
model.add(Embedding(num_words, embed_dim, input_length = X.shape[1]))
model.add(LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2))
model.add(Dense(7,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['categorical_crossentropy'])
print(model.summary())


Y = keras.utils.np_utils.to_categorical(data['Class'].values)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify=Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

batch_size = 32
model.fit(X_train, Y_train, epochs=10, batch_size=batch_size)
    
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1000, 128)         128000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 1379      
Total params: 384,179
Trainable params: 384,179
Non-trainable params: 0
_________________________________________________________________
None
((6002, 1000), (6002, 7))
((1501, 1000), (1501, 7))
Epoch 1/10
6002/6002 [==============================] - 503s 84ms/step - loss: 1.7987 - categorical_crossentropy: 1.7987
Epoch 2/10
6002/6002 [==============================] - 519s 86ms/step - loss: 1.2678 - categorical_crossentropy: 1.2678
Epoch 3/10
6002/6002 [==============================] - 500s 83ms/step - loss: 0.9887 - categorical_c

In [63]:
model.save('emotion_detector')

In [64]:
model_emotion= load_model('emotion_detector')

In [65]:
class_test=model_emotion.predict_classes(X_test)

In [66]:
classes = np.array(range(0,7))
classes

array([0, 1, 2, 3, 4, 5, 6])

In [67]:
accuracy_score(classes[np.argmax(Y_test, axis=1)], class_test)

0.70086608927381744